# API general information

API key:
- Your API key is 1953847a1db7b2070fcd10ee09add91c
- Within the next couple of hours, it will be activated and ready to use
- You can later create more API keys on your account page
- Please, always use your API key in each API call

Endpoint:
- Please, use the endpoint api.openweathermap.org for your API calls
- Example of API call:
api.openweathermap.org/data/2.5/weather?q=London,uk&APPID=1953847a1db7b2070fcd10ee09add91c

Useful links:
- API documentation https://openweathermap.org/api
- Details of your plan https://openweathermap.org/price
- Please, note that 16-days daily forecast and History API are not available for Free subscribers


In [1]:
import requests
import pandas as pd
import numpy as np
import json
import pprint
import time
import string

In [18]:
pd_location = pd.read_excel('population_CH_2021.xlsx')
df = pd_location[1:]
df.head()

,Municipality,Population on 31 December
1,- Zürich,1564662
2,>> Bezirk Affoltern,56370
3,......0001 Aeugst am Albis,1988
4,......0002 Affoltern am Albis,12404
5,......0003 Bonstetten,5632


In [19]:
# throwing away the strange characters
df['Municipality'] = df['Municipality'].str.split(' ', n=1).str[1]

<ipython-input-19-78093223db43>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Municipality'] = df['Municipality'].str.split(' ', n=1).str[1]


In [20]:
print(df['Municipality'])

1                   Zürich
2         Bezirk Affoltern
3          Aeugst am Albis
4       Affoltern am Albis
5               Bonstetten
               ...        
2331          Vendlincourt
2332         Basse-Allaine
2333         Clos du Doubs
2334           Haute-Ajoie
2335            La Baroche
Name: Municipality, Length: 2335, dtype: object


In [21]:
# Renaming columns
df = df.rename(columns={'Population on 31 December':'Population_End2021'})
print(df.shape)
df.head(50)

(2335, 2)


,Municipality,Population_End2021
1,Zürich,1564662
2,Bezirk Affoltern,56370
3,Aeugst am Albis,1988
4,Affoltern am Albis,12404
5,Bonstetten,5632
6,Hausen am Albis,3852
7,Hedingen,3780
8,Kappel am Albis,1290
9,Knonau,2445
10,Maschwanden,641


In [22]:
# Use case-insensitive search to create a mask of rows containing the word "Berzirk" and remove these rows
mask = df['Municipality'].str.contains('Bezirk', case=False)
df = df.drop(df[mask].index)

# Use case-insensitive search to create a mask of rows containing the word "District" and remove these rows
mask2 = df['Municipality'].str.contains('District', case=False)
df = df.drop(df[mask2].index)

# Remove the rows containing the word "Berzirk" using the mask and the drop() method

df.reset_index(drop=True)

df.head(50)

,Municipality,Population_End2021
1,Zürich,1564662
3,Aeugst am Albis,1988
4,Affoltern am Albis,12404
5,Bonstetten,5632
6,Hausen am Albis,3852
7,Hedingen,3780
8,Kappel am Albis,1290
9,Knonau,2445
10,Maschwanden,641
11,Mettmenstetten,5570


In [ ]:
#  Zürich is the largest city in Switzerland with a population of over 428'700
# still missing to delete "(ZH)" and others

In [23]:
list_cities = list(df['Municipality'])
print(len(list_cities))
list_cities

2244


['Zürich',
 'Aeugst am Albis',
 'Affoltern am Albis',
 'Bonstetten',
 'Hausen am Albis',
 'Hedingen',
 'Kappel am Albis',
 'Knonau',
 'Maschwanden',
 'Mettmenstetten',
 'Obfelden',
 'Ottenbach',
 'Rifferswil',
 'Stallikon',
 'Wettswil am Albis',
 'Adlikon',
 'Benken (ZH)',
 'Berg am Irchel',
 'Buch am Irchel',
 'Dachsen',
 'Dorf',
 'Feuerthalen',
 'Flaach',
 'Flurlingen',
 'Andelfingen',
 'Henggart',
 'Humlikon',
 'Kleinandelfingen',
 'Laufen-Uhwiesen',
 'Marthalen',
 'Ossingen',
 'Rheinau',
 'Thalheim an der Thur',
 'Trüllikon',
 'Truttikon',
 'Volken',
 'Stammheim',
 'Bachenbülach',
 'Bassersdorf',
 'Bülach',
 'Dietlikon',
 'Eglisau',
 'Embrach',
 'Freienstein-Teufen',
 'Glattfelden',
 'Hochfelden',
 'Höri',
 'Hüntwangen',
 'Kloten',
 'Lufingen',
 'Nürensdorf',
 'Oberembrach',
 'Opfikon',
 'Rafz',
 'Rorbas',
 'Wallisellen',
 'Wasterkingen',
 'Wil (ZH)',
 'Winkel',
 'Bachs',
 'Boppelsen',
 'Buchs (ZH)',
 'Dällikon',
 'Dänikon',
 'Dielsdorf',
 'Hüttikon',
 'Neerach',
 'Niederglatt',
 '

In [24]:
# Replace with the city and country code you want to get the weather for
#city_names = ["Zermatt", "Andermatt", "Visp", "Oberwald", "Disentis/Mustér"]
API_KEY = "f929f8bae1cf9df1f10a9a2a99661e67"
location = list_cities.copy()
country_code = "CH"
units = "metric"

In [25]:
# Create empyt list for current weather
current_weather = []

for city_name in location:
    # Base URL for OpenWeather API
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name},{country_code}&units={units}&appid={API_KEY}"
       
    # Fetch the current weather data
    response = requests.get(url)
    data = json.loads(response.text)
    current_weather.append(data)

In [237]:
current_weather

[{'coord': {'lon': 8.55, 'lat': 47.3667},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01d'}],
  'base': 'stations',
  'main': {'temp': 14.99,
   'feels_like': 14.01,
   'temp_min': 13.12,
   'temp_max': 16.86,
   'pressure': 1016,
   'humidity': 56},
  'visibility': 10000,
  'wind': {'speed': 4.02, 'deg': 223, 'gust': 7.15},
  'clouds': {'all': 0},
  'dt': 1679480638,
  'sys': {'type': 2,
   'id': 2019255,
   'country': 'CH',
   'sunrise': 1679462742,
   'sunset': 1679506791},
  'timezone': 3600,
  'id': 2657896,
  'name': 'Zurich',
  'cod': 200},
 {'coord': {'lon': 8.4744, 'lat': 47.2731},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01d'}],
  'base': 'stations',
  'main': {'temp': 13.2,
   'feels_like': 12.09,
   'temp_min': 11.51,
   'temp_max': 15.25,
   'pressure': 1015,
   'humidity': 58},
  'visibility': 10000,
  'wind': {'speed': 1.54, 'deg': 350},
  'clouds': {'all': 0},
  'dt': 1679

## Filtering by big cities

This may be needed later on to move the data to the warehouse

In [233]:
# Selecting cities greater than 10K inhabitants
big_cities = df.loc[df['Population_End2021'] > 10000]
big_cities.reset_index(drop=True, inplace=True)
big_cities

,Municipality,Population_End2021
0,Zürich,1564662
1,Affoltern am Albis,12404
2,Bassersdorf,11960
3,Bülach,22193
4,Kloten,20779
...,...,...
234,Vernier,35491
235,Versoix,13280
236,Veyrier,11837
237,Jura,73798


In [199]:
big_cities.drop(columns="index")

,Municipality,Population_End2021
0,Zürich,1564662
1,Bezirk Affoltern,56370
2,Affoltern am Albis,12404
3,Bezirk Andelfingen,32136
4,Bezirk Bülach,158566
...,...,...
317,Jura,73798
318,District de Delémont,39125
319,Delémont,12479
320,District des Franches-Montagnes,10478


In [15]:
df['Municipality'] = df['Municipality'].str.split('Bezirk ').str[1]
df

<ipython-input-15-5d4e38206811>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Municipality'] = df['Municipality'].str.split('Bezirk ').str[1]


,Municipality,Population on 31 December
1,NaN,1564662
2,Affoltern,56370
3,NaN,1988
4,NaN,12404
5,NaN,5632
...,...,...
2331,NaN,559
2332,NaN,1225
2333,NaN,1284
2334,NaN,1073


In [187]:
# Find the index of the duplicated rows
duplicates = big_cities.duplicated(keep=False)
duplicate_indexes = big_cities.loc[duplicates].index

Int64Index([], dtype='int64')

In [182]:
# make list of row indices where 'gmail.com' is in any column
indices = big_cities['Municipality'].where(big_cities['Municipality'].apply(lambda x: x.str.contains('Bezirk'))).dropna(how='all').index.tolist()
indices

AttributeError: 'str' object has no attribute 'str'

In [184]:
print(len(big_cities))
big_cities

322


,index,Municipality,Population_End2021
0,1,Zürich,1564662
1,2,Bezirk Affoltern,56370
2,4,Affoltern am Albis,12404
3,17,Bezirk Andelfingen,32136
4,40,Bezirk Bülach,158566
...,...,...,...
317,2279,Jura,73798
318,2280,District de Delémont,39125
319,2288,Delémont,12479
320,2300,District des Franches-Montagnes,10478
